In [1]:
# imports
import arcpy
from tqdm.notebook import tqdm
from arcpy.sa import *

# ArcGIS setup
arcpy.env.overwriteOutput = True
arcpy.env.extent = "MAXOF"

# set workspace path
workspace = r"E:/fires/fires.gdb/"
arcpy.env.workspace = workspace

# root folder
root_path = r"E:/fires/"

# contains MTBS data 1984-2018 as rasters
data_path = r"E:/fires/mtbs/"

# check out the spatial analyst tools
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

# convert to binary

we only care about the most severe fires (rated as 4 in the rasters). We thus mask out all values that are not zero using [raster algebra](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/raster-calculator.htm) and a [conditional statement](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/con-.htm). Unfortunately, this process is extremely inefficient even on high-end hardware. Classic ArcPy.

In [2]:
# get a list of files to process
raw_mtbs_list = glob.glob(data_path + "*.tif")

# loop through all the files to process
for mtbs_raster in tqdm(raw_mtbs_list):

    # set as a raster
    mtbs_raster = arcpy.Raster(mtbs_raster)
    
    # convert to a binary mask of only "4" (high intensity fires)
    binary_raster = Con(mtbs_raster, 0, 1, "VALUE < 4")
    
    # get a cleaner savename
    save_name = arcpy.Describe(mtbs_raster).basename + "_binary"
    
    # save the binary raster
    binary_raster.save(save_name)

# combine into one map

This produces a map where each pixel value represents the number of times that cell has been severely burned. We use a combination of [cell statistics](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/cell-statistics.htm) to sum the 34 years together, and then [project the raster](https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/project-raster.htm) to make sure everything is in the same projection system for later.

In [4]:
# get a list of rasters
binary_raster_list = arcpy.ListRasters("*binary")

# sum the rasters together
sum_binary_raster = CellStatistics(binary_raster_list, "SUM", "DATA")
print("Rasters combined. Projecting raster...")

# set the save name
save_name = workspace + "binary_sum"

# save and project the results to match the rest of the data
arcpy.ProjectRaster_management(sum_binary_raster, save_name, r"E:/fires/fires.gdb/US_fire_count", "NEAREST")
print("Projection finished!")

Rasters combined. Projecting raster...
Projection finished!


# convert to points
There's [a tool](https://pro.arcgis.com/en/pro-app/latest/tool-reference/conversion/raster-to-point.htm) for this, but with files this large, it's insane and produces a gigantic file that should never ever be touched or opened. Unfortunately we need to do both. The original raster alone was 14GB uncompressed on disk.

In [ ]:
# load the projected raster
projected_binary_raster = arcpy.Raster(workspace + "binary_sum")

# set the save name
save_name = workspace + "binary_sum_points"

# convert to points
arcpy.conversion.RasterToPoint(projected_binary_raster, save_name)

# manually join the points to the fishnet
we already have a fishnet that is perfectly aligned to the rest of the data. We can use [spatial join logic](https://pro.arcgis.com/en/pro-app/latest/tool-reference/analysis/spatial-join.htm) to generate a raster from points and polygons. The ArcPy implementation is sketchy and ugly at best so I've done them manually.

In [ ]:
# get the new files generated manually
fishnet_sum = workspace + "fishnet_join_sum"
fishnet_mean = workspace + "fishnet_join_mean"
fishnet_median = workspace + "fishnet_join_median"

# convert the new fishnest to a raster

fortunately there's [a tool](https://pro.arcgis.com/en/pro-app/latest/tool-reference/conversion/polygon-to-raster.htm) for this already. We set the [snap raster environmental variable](https://pro.arcgis.com/en/pro-app/latest/tool-reference/environment-settings/snap-raster.htm) to ensure the cell sizes and alignments are all the same.

In [ ]:
# set the snap raster setting, this forces alinment and cell sizes to this raster
arcpy.env.snapRaster = r"E:/fires/fires.gdb/US_fire_count"

# define a function to save a few lines of code
def convert_fishnet_to_raster(fishnet, savename):

    # convert from a fishnet back to a raster
    arcpy.conversion.PolygonToRaster(fishnet, 
                                     value_field, 
                                     workspace + savename, 
                                     "MAXIMUM_COMBINED_AREA", 
                                     "", 
                                     "", 
                                     "BUILD")
    
    print(savename, "completed!")

# run all the conversions
convert_fishnet_to_raster( , )
convert_fishnet_to_raster( , )
convert_fishnet_to_raster( , )

In [ ]:
# check in the spatial analyst tools
arcpy.CheckInExtension("Spatial")